In [1]:
import pandas as pd
import math

# **MODEL CREATION** #

''' Hacer orimero que cada palabra de la columna messages sea una linea de un nuevo dataframe.
mirar las palabras mas repetidas.
Asociar valores a las palabras y crear dataframe
en mi dataframe pero copia poner columna por cada tag que encuentre.
despues a partir de aquí asociarle emociones
y construir el modelo.'''

'''NLP LEMMATIZATION
generar nova columna per lemmatization stemming'''

'''Trobar formula del sample minim que necessito per ser representatiu
per agafar el sample random 
llistat emocions dataframe

dels 1000 missatges anar un per un i classificar

entreno sobre els 1000 per trobar el millor model

entreno despres sobre tot el sample i despres ja li poso a tot el meu dataset

fer bootstrapping i fer el m, bootstrapping statistics'''

x boost, random forest, 
amarquescancio@gmail.com

In [2]:
df = pd.read_csv("../data/translation/completo.csv")

In [3]:
df.shape

(118752, 4)

In [4]:
import math

# Datos
N = 118752  # Tamaño total de la población
Z = 1.96    # Valor crítico para un nivel de confianza del 95%
E = 0.05    # Margen de error del 5%
p = 0.5     # Estimación de la proporción

# Fórmula para calcular el tamaño de la muestra
n = (N * Z**2 * p * (1 - p)) / ((E**2 * (N - 1)) + Z**2 * p * (1 - p))

# Redondear hacia arriba ya que no puedes tener fracciones de personas en tu muestra
n = math.ceil(n)

print(f"Min necessary sample: {n}")


Min necessary sample: 383


Even though it says the minimmum sample can be 383 I'm going to catch 1000 to be sure to cover all 3 languages.

audio, image, video = neutral

In [27]:
def get_random_df(df):

    df = df[(df["message"].str.lower() != "\u200eaudio ") & (df["message"].str.lower() != "\u200evideo ")]
    df = df[df["message"].str.lower() != "\u200eimage "] 

    sample_df = df.sample(n=1000, random_state=42, ignore_index=True)
    return sample_df

sdf = get_random_df(df)

In [18]:
sdf.iloc

In [28]:
sdf.sample(30)

,date,time,name,message
41,2018-12-13,12:53:57,Em,Hola amor
771,2021-07-22,12:00:55,Em,‎You deleted this message.
828,2021-07-09,13:53:54,Em,Si o no
152,2017-10-10,18:16:19,Em,Sisi tranqui
407,2023-01-08,19:17:40,Em,Amigaaa perooo quieres que nos veamos?
964,2017-06-16,20:27:10,Em,Dormimos juntos hoy?
940,2019-12-25,20:22:54,Em,A veure que tal
529,2020-12-16,11:53:58,Em,Thxxxx
883,2021-12-10,23:36:21,Em,‎sticker
64,2021-06-06,12:43:18,Em,Seria en espanyol


In [25]:
sdf['classification'] = None

In [31]:
sdf.to_csv("../data/translation/to_classify.csv")

In [39]:
emotions= [{"sadness":1},
           {"joy":2},
           {"fear":3},
           {"anger":4},
           {"surprise":5},
           {"defensive":6},
           {"nostalgia":7},
           {"hapiness":8},
           {"stressed":9},
           {"expectant":10},
           {"insecure":11},
           {"assertivity":12},
           {"neutral":13},
           {"excitement":14},
           {"comprehensive":15},
           {"helpless":16},
           {"love":17},
           {"deception":18},
           {"doubtful":19},
           {"picarona":20}]

In [ ]:
11